In [2]:
from molt5_dataset import Mol2CaptionDataset
import torch
from torch.utils.data import DataLoader
from molxpt_tokenizer import  MolxptTokenizer
# 假设 raw_folder 和 pro_folder 是你的数据文件夹路径，mode 是你的数据模式
import weaviate
import re
raw_folder = '/Users/wangjuede/Documents/mol2cap_data'
mode = 'train'  # 可以是 'train', 'val' 或 'test'

# 实例化数据集


In [3]:
tokenizer = MolxptTokenizer.from_pretrained("molxpt_ckpt", use_fast=False)

dataset = Mol2CaptionDataset(raw_folder, mode, tokenizer=tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BioGptTokenizer'. 
The class this function is called from is 'MolxptTokenizer'.


In [2]:
for i in range(5):
    if i < len(dataset):
        item = dataset[i]  # 获取第 i 个数据项
        print(item[1])  # 打印编码后的 caption
        

tensor([[    2,    28,  1494,    30,    47, 21079,    18, 11514,    17, 20806,
         14861, 15297,   808,    32,    30,    10,  6209,  1679,     9,   514,
          3899,    93,    18,  1905,   404,    11,  1801,   474,    17,   106,
         13820, 19811,    18,   311,  1683,    11,   435,  1683,    11,  1530,
           463,    17,   106, 20806, 14861, 17871,   244,    11,   273,    31,
           866, 14384,     9,    10, 36023,    85,    61,   356,   299,    48,
           596,  5870,     8,   242,    30,    22,  6209,  1679,     9,    47,
           514,  3899,    93,    18,  1905,   404,    11,  1801,   474,    17,
           106, 13820, 19811,    18,   311,  1683,    11,   435,  1683,    11,
          1530,   463,    17,   106, 20806, 14861, 17871,   244,     8]])
tensor([[    2,    28,  1494,    30,    10,   953,  5857,     9,   653,  3487,
         22584,    23,   652,  4159,   457,  1530,  1277,  4252,  8362,   494,
            11, 25363,  4311,  1462,   971,  3307,    13,

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("molxpt_ckpt")
# 访问嵌入层
embedding_layer = model.biogpt.embed_tokens

In [5]:
print(dataset[1])

('[125Te]', tensor([[    2,    28,  1494,    30,    10,   953,  5857,     9,   653,  3487,
         22584,    23,   652,  4159,   457,  1530,  1277,  4252,  8362,   494,
            11, 25363,  4311,  1462,   971,  3307,    13,   918,  4616,    71,
            37,    65,     8]]), 'The molecule is the stable isotope of tellurium with relative atomic mass 124.904425, 71.4 atom percent natural abundance and nuclear spin 1/2.')


In [11]:
print(model(torch.tensor(dataset[1][1])).hidden_states[-2])

/var/folders/kf/myby0xcs14j14p4sg5dgxnbc0000gn/T/ipykernel_62884/1424585921.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(model(torch.tensor(dataset[1][1])).hidden_states[-2])


TypeError: 'NoneType' object is not subscriptable

In [11]:
from transformers import BioGptModel, BioGptTokenizer

# 加载预训练模型和分词器
model = BioGptModel.from_pretrained('molxpt_ckpt')


In [10]:
#连接数据库
import weaviate

client = weaviate.Client(
  url="https://rag-zm3s034w.weaviate.network",
  auth_client_secret=weaviate.auth.AuthApiKey(api_key="VwYY6pZ6pJYvCgj4E05yLzEvbdhFuOJ4L0IA"),
)
client.is_ready()

True

In [17]:
class_obj = {
    "class": "Compound_caption",
    "vectorizer": "none",
}

# Add the class to the schema
client.schema.create_class(class_obj)



In [ ]:
print(model(torch.tensor(dataset[1][1])))

In [9]:
updated_dataset = []
with torch.no_grad():
# 遍历原始数据集中的每个元组
    for item in dataset:
        # 对于每个元组，创建一个新的元组，其中第二个元素是使用嵌入层生成的向量
        # 假设 item[1] 是一个可以被嵌入层处理的列表或张量
        embedded_item = embedding_layer(item[1])
        pooling_item = torch.mean(embedded_item,dim=1)
        
        # 将原始的第一个元素和新生成的嵌入向量作为第二个元素组合成新的元组
        new_item = (item[0], pooling_item, item[2])
        
        # 将新的元组添加到更新后的数据集列表中
        updated_dataset.append(new_item)
print(updated_dataset[1])

('[125Te]', tensor([[-0.0068,  0.0009, -0.0018,  ...,  0.0103, -0.0030,  0.0130]]), 'The molecule is the stable isotope of tellurium with relative atomic mass 124.904425, 71.4 atom percent natural abundance and nuclear spin 1/2.')


In [17]:
print(updated_dataset[2][1].size())

torch.Size([1, 1024])


In [ ]:

# Configure a batch process
client.batch.configure(batch_size=5)  # Configure batch
for i, (label, tensor,caption) in enumerate(updated_dataset):
    if i >= 11535:
        try:
            # 尝试从 Weaviate 获取对象
            existing_object = client.get('Compound_caption', label, ['Smiles', 'caption'])
            print(f"Object {i+1} already exists. Skipping...")
        except Exception as e:
            # 如果在获取时发生异常（例如，对象不存在），则处理异常并继续导入
            print(f"Importing: {i+1}")
            vector = tensor.view(-1).tolist()  # Flatten the tensor and convert to list
    
            # 创建 Weaviate 需要的对象属性字典
            properties = {
                "Smiles": label,
                "caption": caption,
                # 其他属性...
            }
    
            # 使用批量客户端导入新对象
            with client.batch as batch:
                # 添加数据对象到批量请求中
                batch.add_data_object(properties, "Compound_caption", vector=vector)

# 提交批量导入
client.batch.flush()


Inference 从此开始

In [11]:
molxpt_tokenizer = MolxptTokenizer.from_pretrained("molxpt_ckpt", use_fast=False)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BioGptTokenizer'. 
The class this function is called from is 'MolxptTokenizer'.


In [17]:
def retro(vector):
    client = weaviate.Client(
      url="https://rag-zm3s034w.weaviate.network",
      auth_client_secret=weaviate.auth.AuthApiKey(api_key="VwYY6pZ6pJYvCgj4E05yLzEvbdhFuOJ4L0IA"),
    )
    client.is_ready()
    
    retrieval = client.query.get(
    "Compound_caption",["smiles","caption"]).with_near_vector(
    vector
    ).with_limit(2).do()
    return retrieval
    
def generate_mol(input_text, prompt_template, tokenizer=molxpt_tokenizer):
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    input_embedded = torch.mean(embedding_layer(input_ids),dim=1)
        
    retro_vec = {
        "vector": input_embedded.view(-1).tolist(),
    }
    
    retrieval = retro(retro_vec)
    

    inputs = prompt_template.format(
                                    caption_1 = retrieval["data"]["Get"]["Compound_caption"][0]["caption"],
                                    caption_2 =retrieval["data"]["Get"]["Compound_caption"][1]["caption"],
                                    smiles_1 = retrieval["data"]["Get"]["Compound_caption"][0]["smiles"],
                                    smiles_2 = retrieval["data"]["Get"]["Compound_caption"][1]["smiles"],
                                    caption = input_text,
                                    )
    
    input_ids = tokenizer(inputs, return_tensors="pt").input_ids
    output = model.generate(
        input_ids,
        max_new_tokens=50,
        temperature=0.75,
        do_sample=True,
    )
    s = tokenizer.decode(output[0])
    start_index = s.find("SMILES of this molecule is") + len("SMILES of this molecule is")#定位
    #正则 
    substring = s[start_index:]
    match = re.search(r'<start-of-mol>(.*?)<end-of-mol>', substring)
    if match:
        smiles = match.group(1)  # 这是匹配到的SMILES字符串
    else:
        smiles = "No SMILES found"
    return smiles,inputs

def generate_mol_molxpt(input_text, prompt_template, tokenizer=molxpt_tokenizer):
    inputs = prompt_template.format(caption = input_text,
                                    )
    #print(inputs)
    input_ids = molxpt_tokenizer(inputs, return_tensors="pt").input_ids
    output = model.generate(
        input_ids,
        max_new_tokens=50,
        temperature=0.75,
        do_sample=True,
    )
    #print(output)
    s = molxpt_tokenizer.decode(output[0])
    
    all_matches = re.findall(r'<start-of-mol>(.*?)<end-of-mol>', s)

    # 检查是否有足够的匹配项
    if len(all_matches) > 1:
        # 获取第二个匹配项
        smiles = all_matches[1]
        #print(2)
        
    elif len(all_matches) == 1:
        smiles = all_matches[0]
        #print(1)
    else:
        smiles = "No SMILES found"
    return smiles
    

In [18]:
prompt = "Here are two template molecule. First. {caption_1} SMILES of this molecule is <start-of-mol>{smiles_1}<end-of-mol>. Second. {caption_2} SMILES of this molecule is <start-of-mol>{smiles_2}<end-of-mol>. Now. {caption} SMILES of this molecule is "

prompt_molxpt = "{caption}.The compound is <start-of-mol>"

input_text = "The molecule is a bile acid taurine conjugate of ursocholic acid. It has a role as a human metabolite and a rat metabolite. It derives from an ursocholic acid. It is a conjugate acid of a tauroursocholate."
print(generate_mol(input_text,prompt))


('C[C@H](CCC(=O)NCCS(=O)(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2CC[C@H]4[C@@]3(CC[C@H](C4)O)C)C', 'Here are two template molecule. First. The molecule is the bile acid taurine conjugate of lithocholic acid. It has a role as a human metabolite. It is a monocarboxylic acid amide and a bile acid taurine conjugate. It derives from a lithocholic acid. It is a conjugate acid of a taurolithocholate. SMILES of this molecule is <start-of-mol>C[C@H](CCC(=O)NCCS(=O)(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2CC[C@H]4[C@@]3(CC[C@H](C4)O)C)C<end-of-mol>. Second. The molecule is a bile acid taurine conjugate of cholic acid that usually occurs as the sodium salt of bile in mammals. It has a role as a human metabolite. It is an amino sulfonic acid and a bile acid taurine conjugate. It derives from a cholic acid. It is a conjugate acid of a taurocholate. SMILES of this molecule is <start-of-mol>C[C@H](CCC(=O)NCCS(=O)(=O)O)[C@H]1CC[C@@H]2[C@@]1([C@H](C[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)O)

In [13]:
generate_mol_molxpt(input_text, prompt_molxpt)

'ocsbcnconosococc.Isosnbonbo.Isonsococc.Isconcoosoco..copons'

 Evaluation

In [6]:
import json

mode = "test"
test_data = Mol2CaptionDataset(raw_folder, mode, tokenizer=molxpt_tokenizer)
json_file_path = '/Users/wangjuede/Downloads/molxpt/molxpt_code/similarity_results.json'
 
# 读取JSON文件并将其转换为字典
with open(json_file_path, 'r') as file:
    results = json.load(file)

In [25]:
from rdkit import DataStructs,Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors,Fingerprints
import warnings

warnings.filterwarnings('ignore')

def Fingerprint_sim(mode,std_mol,gen_mol):
    if mode == "maccs":
        std_fp = rdMolDescriptors.GetMACCSKeysFingerprint(std_mol)
        gen_fp = rdMolDescriptors.GetMACCSKeysFingerprint(gen_mol)
    elif mode == "rdk":
        std_fp = Chem.RDKFingerprint(std_mol)
        gen_fp = Chem.RDKFingerprint(gen_mol)
    elif mode == "morgan":
        std_fp = AllChem.GetMorganFingerprintAsBitVect(std_mol, radius=2)
        gen_fp = AllChem.GetMorganFingerprintAsBitVect(gen_mol, radius=2)
    tanimoto_sim = DataStructs.TanimotoSimilarity(std_fp, gen_fp)
    return tanimoto_sim
    

i=0
for smiles,_,caption in test_data:
    if results.get(smiles) is not None:
        
        continue
    else:
        try:
            smiles_gen = generate_mol(caption,prompt)
            
            std_mol = Chem.MolFromSmiles(smiles)
            gen_mol = Chem.MolFromSmiles(smiles_gen)
            #有效性检验
            if not std_mol or not gen_mol:
                results[smiles] = {"error": "Invalid molecule"}
                continue
            tanimoto_sim_maccs = Fingerprint_sim("maccs",std_mol,gen_mol)
            tanimoto_sim_rdk = Fingerprint_sim("rdk",std_mol,gen_mol)
            tanimoto_sim_morgan = Fingerprint_sim("morgan",std_mol,gen_mol)
          
            results[smiles] = { "gen_mol" : gen_mol,
                                "tanimoto_sim_maccs": tanimoto_sim_maccs,
                                "tanimoto_sim_rdk": tanimoto_sim_rdk,
                                "tanimoto_sim_morgan": tanimoto_sim_morgan
            }
        except:
            continue
    i += 1
    print(i)
results_json = json.dumps(results, indent=4)

# 将JSON字符串存储到文件
with open('similarity_results.json', 'w') as json_file:
    json_file.write(results_json)

print('相似度结果已成功存储为JSON格式。')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


[11:54:22] WARNING: not removing hydrogen atom without neighbors


578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
相似度结果已成功

In [ ]:
from rdkit import DataStructs,Chem
from rdkit.Chem import AllChem

from rdkit.Chem import rdMolDescriptors,Fingerprints
import warnings

warnings.filterwarnings('ignore')

def Fingerprint_sim(mode,std_mol,gen_mol):
    if mode == "maccs":
        std_fp = rdMolDescriptors.GetMACCSKeysFingerprint(std_mol)
        gen_fp = rdMolDescriptors.GetMACCSKeysFingerprint(gen_mol)
    elif mode == "rdk":
        std_fp = Chem.RDKFingerprint(std_mol)
        gen_fp = Chem.RDKFingerprint(gen_mol)
    elif mode == "morgan":
        std_fp = AllChem.GetMorganFingerprintAsBitVect(std_mol, radius=2)
        gen_fp = AllChem.GetMorganFingerprintAsBitVect(gen_mol, radius=2)
    tanimoto_sim = DataStructs.TanimotoSimilarity(std_fp, gen_fp)
    return tanimoto_sim
    
results = {}
i=0
for smiles,_,caption in test_data:
    if results.get(smiles) is not None:
        i+=1
        print("exist,{}".format(i))
        continue
    else:
        try:
            smiles_gen = generate_mol_molxpt(caption,prompt_molxpt)
            
            std_mol = Chem.MolFromSmiles(smiles)
            gen_mol = Chem.MolFromSmiles(smiles_gen)
            #有效性检验
            if not std_mol or not gen_mol:
                results[smiles] = {"error": "Invalid molecule"}
                continue
            tanimoto_sim_maccs = Fingerprint_sim("maccs",std_mol,gen_mol)
            tanimoto_sim_rdk = Fingerprint_sim("rdk",std_mol,gen_mol)
            tanimoto_sim_morgan = Fingerprint_sim("morgan",std_mol,gen_mol)
          
            results[smiles] = { "gen_mol" : smiles_gen,
                                "tanimoto_sim_maccs": tanimoto_sim_maccs,
                                "tanimoto_sim_rdk": tanimoto_sim_rdk,
                                "tanimoto_sim_morgan": tanimoto_sim_morgan
            }
            i+=1
            print(i)
        except:
            print("error")
            i+=1
            print(i)
            continue
        
results_json = json.dumps(results, indent=4)

# 将JSON字符串存储到文件
with open('similarity_results_molxpt.json', 'w') as json_file:
    json_file.write(results_json)

print('相似度结果已成功存储为JSON格式。')

In [19]:
results_json = json.dumps(results, indent=4)
print(1)
# 将JSON字符串存储到文件
with open('similarity_results_molxpt.json', 'w') as json_file:
    json_file.write(results_json)

print('相似度结果已成功存储为JSON格式。')

TypeError: Object of type Mol is not JSON serializable

In [23]:
print(results[])

KeyError: 1

In [29]:
with open('similarity_results_molxpt.json', 'r') as json_file:
    data = json.load(json_file)
    
sum_similarity_maccs = 0
sum_similarity_rdk = 0
sum_similarity_morgan = 0
totally_similar = []
i = 0
# 遍历化合物数据
for compound, similarities in data.items():
    # 计算平均相似度
    if "tanimoto_sim_maccs" in similarities:
        i+=1
        sum_similarity_maccs += similarities["tanimoto_sim_maccs"]
        sum_similarity_rdk += similarities["tanimoto_sim_rdk"]
        sum_similarity_morgan += similarities["tanimoto_sim_morgan"]
        if similarities["tanimoto_sim_maccs"] == 1.0 and similarities["tanimoto_sim_rdk"] and similarities["tanimoto_sim_morgan"]:
            totally_similar.append(compound)
        #print(1)
                        
average_similarity_maccs = sum_similarity_maccs / i
average_similarity_rdk = sum_similarity_rdk / i
average_similarity_morgan = sum_similarity_morgan / i
average_sim = {
                "tanimoto_sim_maccs": average_similarity_maccs, 
                "tanimoto_sim_rdk": average_similarity_rdk, 
                "tanimoto_sim_morgan": average_similarity_morgan
}
# 输出结果
print(average_sim)
print("Totally similar molecule:",len(totally_similar))
print("Valid molecule:",i)

{'tanimoto_sim_maccs': 0.1813161385586092, 'tanimoto_sim_rdk': 0.0794704567512316, 'tanimoto_sim_morgan': 0.05672881185065994}
Totally similar molecule: 0
Valid molecule: 1778


In [30]:
with open('similarity_results.json', 'r') as json_file:
    data = json.load(json_file)
    
sum_similarity_maccs = 0
sum_similarity_rdk = 0
sum_similarity_morgan = 0
totally_similar = []
i = 0
# 遍历化合物数据
for compound, similarities in data.items():
    # 计算平均相似度
    if "tanimoto_sim_maccs" in similarities:
        i+=1
        sum_similarity_maccs += similarities["tanimoto_sim_maccs"]
        sum_similarity_rdk += similarities["tanimoto_sim_rdk"]
        sum_similarity_morgan += similarities["tanimoto_sim_morgan"]
        if similarities["tanimoto_sim_maccs"] == 1.0 and similarities["tanimoto_sim_rdk"] and similarities["tanimoto_sim_morgan"]:
            totally_similar.append(compound)
        #print(1)
                        
average_similarity_maccs = sum_similarity_maccs / i
average_similarity_rdk = sum_similarity_rdk / i
average_similarity_morgan = sum_similarity_morgan / i
average_sim = {
                "tanimoto_sim_maccs": average_similarity_maccs, 
                "tanimoto_sim_rdk": average_similarity_rdk, 
                "tanimoto_sim_morgan": average_similarity_morgan
}
# 输出结果
print(average_sim)
print("Totally similar molecule:",len(totally_similar))
print("Valid molecule:",i)

{'tanimoto_sim_maccs': 0.7899117205304005, 'tanimoto_sim_rdk': 0.6937007764215024, 'tanimoto_sim_morgan': 0.5697404357824769}
Totally similar molecule: 328
Valid molecule: 1473


In [13]:
input_text = "The molecule is a bile acid taurine conjugate of ursocholic acid. It has a role as a human metabolite and a rat metabolite. It derives from an ursocholic acid. It is a conjugate acid of a tauroursocholate."
input_ids = molxpt_tokenizer(input_text, return_tensors="pt").input_ids
input_embedded = torch.mean(embedding_layer(input_ids),dim=1)
    
retro_vec = {
    "vector": input_embedded.view(-1).tolist(),
}

#print(len(retro_vec["vector"]))
#print(input_text)


In [14]:
print(retrieval["data"]["Get"]["Compound_caption"][0]["smiles"])
print(retrieval["data"]["Get"]["Compound_caption"][0]["caption"])


NameError: name 'retrieval' is not defined

In [50]:
prompt = "{caption}.The compound is <start-of-mol>"
inputs = prompt.format(caption = input_text,
                                )
print(inputs)
print()
input_ids = molxpt_tokenizer(inputs, return_tensors="pt").input_ids
output = model.generate(
    input_ids,
    max_new_tokens=50,
    temperature=0.75,
    do_sample=True,
)
#print(output)
s = molxpt_tokenizer.decode(output[0])
print(s)


The molecule is a bile acid taurine conjugate of ursocholic acid. It has a role as a human metabolite and a rat metabolite. It derives from an ursocholic acid. It is a conjugate acid of a tauroursocholate..The compound is <start-of-mol>
</s> <start-of-mol>ocsbcnconosococc.Isosnbonbo.Isonsococc.Isconcoosoco..copons<end-of-mol>. </s>


In [ ]:
generate_mol(input_text,prompt_template=prompt)

In [18]:
prompt_template = "Here are two template molecule. First. {caption_1} SMILES of this molecule is <start-of-mol>{smiles_1}<end-of-mol>. Second. {caption_2} SMILES of this molecule is <start-of-mol>{smiles_2}<end-of-mol>. Now. {caption} SMILES of this molecule is "


inputs = prompt_template.format(caption_1 = retrieval["data"]["Get"]["Compound_caption"][0]["caption"],
                                caption_2 = retrieval["data"]["Get"]["Compound_caption"][1]["caption"],
                                smiles_1 = retrieval["data"]["Get"]["Compound_caption"][0]["smiles"],
                                smiles_2 = retrieval["data"]["Get"]["Compound_caption"][1]["smiles"],
                                caption = input_text,
                                )

input_ids = molxpt_tokenizer(inputs, return_tensors="pt").input_ids
output = model.generate(
    input_ids,
    max_new_tokens=50,
    temperature=0.75,
    do_sample=True,
)

print(output)
s = molxpt_tokenizer.decode(output[0])
#s = s[len(inputs):].strip()
print(s)

tensor([[    2,   559,    46,   100,  6247,  1494,     8,  3893,     8,    28,
          1494,    30,    22,  2708,     3,   106,  6399,  4094, 20103,  4414,
           273,    31,   866, 14384,     9,    10, 18870,  2791,   157,     9,
           611,   268,  5668,     3,   106,  6399,   141,     3,   106,  6399,
         23091,   603,     3,   106,  6399,  3452,   182,     3,   106,  6399,
           607,   866, 11081,     3,   106,  6399,   574,     3,   106,  6399,
           427,     3,   106,  6399,   965,    61,   356,   299,    48,   596,
          5870,     8,   242,    81,    22,   173,    40,    22,   910,  2891,
             8,   242, 32895,    41,    22,   611,   268,  5668,     3,   106,
          6399,   574,     3,   106,  6399,   427,     3,   106,  6399,   965,
            18,  3340,    17,     8,   242,    30,    22,  6209,  1679,     9,
            22,   611,   268,  5668,     3,   106,  6399,   141,     3,   106,
          6399, 23091,   603,     3,   106,  6399,  